In [478]:
import datetime
import pandas as pd
import numpy as np

In [479]:
data_folder = "./data/"

df_legislaturas = pd.read_csv(data_folder + "df_legislaturas.csv")
df_parlamentares_por_legislatura = pd.read_csv(
    data_folder + "df_parlamentares_por_legislatura.csv"
)
df_parl_detalhes = pd.read_csv(data_folder + "df_parl_detalhes.csv", dtype={"cpf": str})
df_legis_lideres = pd.read_csv(data_folder + "df_legis_lideres.csv")
df_legis_mesa = pd.read_csv(data_folder + "df_legis_mesa.csv")
df_cargos_externos = pd.read_csv(data_folder + "df_cargos_externos.csv")
df_historico = pd.read_csv(data_folder + "df_historico.csv")
df_pareceres = pd.read_csv(data_folder + "df_pareceres.csv")
df_pareceres_detalhes = pd.read_csv(data_folder + "df_pareceres_detalhes.csv", low_memory=False)
df_orgaos_membros = pd.read_csv(data_folder + "df_orgaos_membros.csv")
df_tse_candidatos = pd.read_csv(data_folder + "df_tse_candidatos.csv")

In [480]:
df_legislaturas["dataInicio"] = pd.to_datetime(df_legislaturas["dataInicio"])
df_legislaturas["dataFim"] = pd.to_datetime(df_legislaturas["dataFim"])

df_legislaturas["dataInicioYear"] = df_legislaturas["dataInicio"].dt.year
df_legislaturas["dataFimYear"] = df_legislaturas["dataFim"].dt.year

df_historico["dataHora"] = pd.to_datetime(df_historico["dataHora"])
# tratar cpfs
df_tse_candidatos['cpf'] = df_tse_candidatos['cpf'].astype(str).str.zfill(11)

In [481]:
df_parlamentares_por_legislatura = df_parlamentares_por_legislatura[
    ["id", "nome", "idLegislatura"]
].drop_duplicates()

# dimensão posição (mesa)

In [482]:
columns = ["id", "idLegislatura", "titulo", "codTitulo"]
group_columns = ["id", "idLegislatura", "titulo"]
df_legis_g = df_legis_mesa[columns].groupby(group_columns).count().reset_index()

In [483]:
posicao_mapper = {
    "Presidente": "mesa_1",
    "1º Vice-Presidente": "mesa_2",
    "2º Vice-Presidente": "mesa_2",
    "1º Secretário": "mesa_2",
    "2º Secretário": "mesa_2",
    "3º Secretário": "mesa_2",
    "4º Secretário": "mesa_2",
    # "2º Suplente de Secretário": "mesa_2",
    # "3º Suplente de Secretário": "mesa_2",
    # "4º Suplente de Secretário": "mesa_2",
    # "1º Suplente de Secretário": "mesa_2",
}

df_legis_g["posicao"] = df_legis_g["titulo"].apply(lambda x: posicao_mapper.get(x))

In [484]:
df_legis_final = df_legis_g.join(pd.get_dummies(df_legis_g["posicao"]))
# df_legis_final = df_legis_final[['id', 'idLegislatura', 'mesa_1', 'mesa_2']].reset_index(drop=True)
df_legis_final = (
    df_legis_final[["id", "idLegislatura", "mesa_1", "mesa_2"]]
    .groupby(["id", "idLegislatura"])
    .sum()
    .reset_index()
)

# dimensão posição (colégio de lideres)

In [485]:
df_legis_lideres_final = (
    df_legis_lideres[df_legis_lideres["titulo"].isin(["Líder", "Presidente"])]
    .drop_duplicates(["parlamentar.id", "parlamentar.idLegislatura", "titulo"])
    .groupby(["parlamentar.id", "parlamentar.idLegislatura"])
    .count()
    .reset_index()
    .rename(
        {
            "parlamentar.id": "id",
            "parlamentar.idLegislatura": "idLegislatura",
            "titulo": "pos_lider",
        },
        axis=1,
    )[["id", "idLegislatura", "pos_lider"]]
)

# dimensão presidencia de comissão

In [486]:
pos_comissao_mapper = {
    # "Coordenador": "pos_comiss_pr",
    # "Coordenadora": "pos_comiss_pr",
    # "Coordenador-Geral'": "pos_comiss_pr",
    "Presidente": "pos_comiss_pr",
}

In [487]:
df_orgaos_membros["titulo_trat"] = df_orgaos_membros["titulo"].apply(
    lambda x: pos_comissao_mapper.get(x)
)

In [488]:
df_orgaos_membros_final = (
    df_orgaos_membros[~df_orgaos_membros["titulo_trat"].isnull()]
    .drop_duplicates(["id_org", "id", "idLegislatura"])[
        ["id_org", "id", "idLegislatura"]
    ]
    .groupby(["id", "idLegislatura"])
    .count()
    .reset_index()
    .rename({"id_org": "pos_comiss_pr"}, axis=1)
)

# pareceres de relator

In [489]:
df_pareceres_merge = df_pareceres[["id", "id_parl"]].merge(
    df_pareceres_detalhes, on="id"
)

## encontrar legislatura por data de apresentação

In [490]:
df_pareceres_merge["dataApresentacao"] = pd.to_datetime(
    df_pareceres_merge["dataApresentacao"]
)

In [491]:
def find_legislatura(date):
    legislatura = df_legislaturas[
        (df_legislaturas["dataInicio"] <= date) & (df_legislaturas["dataFim"] >= date)
    ]
    if not legislatura.empty:
        return int(legislatura.iloc[0]["id"])
    return None

In [492]:
df_pareceres_merge["idLegislatura"] = df_pareceres_merge["dataApresentacao"].apply(
    find_legislatura
)
df_pareceres_merge["idLegislatura"] = (
    df_pareceres_merge["idLegislatura"].fillna(-1).astype("int64")
)

In [493]:
df_pareceres_final = (
    df_pareceres_merge[["id", "id_parl", "idLegislatura"]]
    .groupby(["id_parl", "idLegislatura"])
    .count()
    .reset_index()
    .rename({"id": "relatorias", "id_parl": "id"}, axis=1)
)

In [515]:
df_pareceres_final['relatorias_ln'] = np.log(df_pareceres_final['relatorias'])

# mandatos externos

In [494]:
df_mandatos_externos = pd.merge(
    df_parlamentares_por_legislatura,
    df_legislaturas[["id", "dataInicioYear"]],
    left_on="idLegislatura",
    right_on="id",
    how="left",
)

df_mandatos_externos.drop(columns=["id_y"], inplace=True)
df_mandatos_externos.rename(columns={"id_x": "id"}, inplace=True)

df_mandatos_externos["mand_dep_estadual"] = 0
df_mandatos_externos["mand_ver"] = 0
df_mandatos_externos["mand_sen"] = 0


def update_dim_columns(row, df_cargos, df_legislaturas):
    leg_data_inicio = df_legislaturas.loc[
        df_legislaturas["id"] == row["idLegislatura"], "dataInicio"
    ].dt.year.values[0]

    relevant_cargos = df_cargos[
        (df_cargos["id_parl"] == row["id"]) & (df_cargos["anoInicio"] < leg_data_inicio)
    ]

    for _, cargo in relevant_cargos.iterrows():
        if cargo["cargo_trat"] == "mand_dep_estadual":
            row["mand_dep_estadual"] += 1
        elif cargo["cargo_trat"] == "mand_ver":
            row["mand_ver"] += 1
        elif cargo["cargo_trat"] == "mand_sen":
            row["mand_sen"] += 1

    return row

In [495]:
def find_mandato(x):
    texto = x.lower()
    if "suplente" in texto:
        return None
    elif "estadual" in texto:
        return "mand_dep_estadual"
    elif "veread" in texto:
        return "mand_ver"
    elif "senad" in texto:
        return "mand_sen"
    else:
        return None
    
df_cargos_externos['cargo_trat'] = df_cargos_externos['cargo'].apply(lambda x: find_mandato(x))
df_cargos_externos['anos_mandato'] = df_cargos_externos['anoFim'] - df_cargos_externos['anoInicio']

In [496]:
df_cargos_externos_clean = df_cargos_externos[
    ~df_cargos_externos["cargo_trat"].isnull()
]

df_mandatos_externos = df_mandatos_externos.apply(
    update_dim_columns, axis=1, args=(df_cargos_externos_clean, df_legislaturas)
)

df_mandatos_externos.drop(columns=["dataInicioYear", "nome"], inplace=True)

# dim tempo de atuação na camara

In [497]:
def compute_period_in_ex(row, df_historico, df_legislaturas):
    id_parl = row["id"]
    id_legis = row["idLegislatura"]
    
    current_legis_row = df_legislaturas[df_legislaturas["dataFim"] > datetime.datetime.today()].iloc[0]
    id_legislatura_atual = current_legis_row["id"]
    
    legis_row = df_legislaturas[df_legislaturas["id"] == id_legis].iloc[0]
    date_init_legis = legis_row["dataInicio"]
    date_end_legis = legis_row["dataFim"]

    df_hist_parl = df_historico[
        (df_historico["id"] == id_parl)
        & (df_historico["idLegislatura"] == id_legis)
        & (df_historico["dataHora"].dt.floor('D') >= np.datetime64(date_init_legis, 'D'))
        & (df_historico["dataHora"].dt.floor('D') <= np.datetime64(date_end_legis, 'D'))
    ].sort_values("dataHora")

    period_in_ex = 0
    last_data_hora_ex = None
    in_ex = False
    
    for _, row_hist in df_hist_parl.iterrows():
        status = row_hist["situacao"]
        data_hora = row_hist["dataHora"]

        if status == "Exercício" or (str(status) == 'nan' and id_legis < 50) :
            if not in_ex:
                last_data_hora_ex = data_hora
                in_ex = True
        else:
            if in_ex and status != "Convocado":
                period_in_ex += (data_hora - last_data_hora_ex).days
                in_ex = False

    if id_legis == id_legislatura_atual:
        total_days = (datetime.datetime.today() - date_init_legis).days - 32
        if in_ex:
            period_in_ex += (datetime.datetime.today() - last_data_hora_ex).days
    else:
        total_days = (date_end_legis - date_init_legis).days


    if in_ex and period_in_ex == 0:
        period_in_ex = (data_hora - last_data_hora_ex).days


    percent_period_in_ex = round(period_in_ex / total_days, 2)
    row["tempo_atuacao_dias"] = period_in_ex
    row["tempo_atuacao_percent"] = percent_period_in_ex
    # row["last_data_hora_ex"] = last_data_hora_ex
    # row["last_data_hora"] = data_hora
    # row["in_ex"] = in_ex
    return row

In [498]:
df_tempo_atuacao = df_parlamentares_por_legislatura.copy()

df_tempo_atuacao = df_tempo_atuacao.apply(
    compute_period_in_ex, axis=1, args=(df_historico, df_legislaturas)
)

df_tempo_atuacao.drop(columns=["nome"], inplace=True)

# dim fidelidade

In [499]:
df_fidelidade = pd.merge(
    df_parlamentares_por_legislatura,
    df_legislaturas[["id", "dataInicioYear", "dataFimYear"]],
    left_on="idLegislatura",
    right_on="id",
    how="left",
)

df_fidelidade.drop(columns=["id_y"], inplace=True)
df_fidelidade.rename(columns={"id_x": "id"}, inplace=True)

df_fidelidade = df_fidelidade.merge(df_parl_detalhes[["id", "cpf"]], on="id")

df_fidelidade['fid_municipais'] = 0
df_fidelidade['fid_gerais'] = 0

In [500]:
def update_fid_columns(row, df_tse):
    cargos_municipais = ["vereador", "prefeito", "vice-prefeito"]
    cargos_gerais_congresso = [
        # "1º suplente senador",
        "senador",
        "deputado federal",
        # "2º suplente senador",
        # "1º suplente",
        # "2º suplente",
    ]
    eleicoes_municipais = df_tse[
        (df_tse["cargo"].isin(cargos_municipais))
        & (df_tse["ano"] > row["dataInicioYear"])
        & (df_tse["ano"] < row["dataFimYear"])
    ]

    eleicoes_cn = df_tse[
        (df_tse["cargo"].isin(cargos_gerais_congresso))
        & (df_tse["ano"] > row["dataInicioYear"])
        & (df_tse["ano"] < row["dataFimYear"])
    ]

    
    if row['cpf'] not in eleicoes_municipais['cpf'].values:
        row['fid_municipais'] = 1

    if row['cpf'] in eleicoes_cn['cpf'].values:
        row['fid_gerais'] = 1

    return row

In [501]:
df_fidelidade = df_fidelidade.apply(update_fid_columns, axis=1, args=(df_tse_candidatos,))
df_fidelidade.drop(columns=["dataInicioYear", "dataFimYear", "nome", "cpf"], inplace=True)

# merge dims

In [516]:
df_final = (
    df_parlamentares_por_legislatura[["id", "idLegislatura"]]
    .merge(
        df_parl_detalhes[
            [
                "id",
                "ultimoStatus.nome",
                "ultimoStatus.siglaPartido",
                "sexo",
                "dataNascimento",
                "escolaridade",
            ]
        ],
        on="id",
        how="left",
    )
    .merge(df_tempo_atuacao, on=["id", "idLegislatura"], how="left")
    .merge(df_legis_final, on=["id", "idLegislatura"], how="left")
    .merge(df_legis_lideres_final, on=["id", "idLegislatura"], how="left")
    .merge(df_orgaos_membros_final, on=["id", "idLegislatura"], how="left")
    .merge(df_pareceres_final, on=["id", "idLegislatura"], how="left")
    .merge(df_mandatos_externos, on=["id", "idLegislatura"], how="left")
    .merge(df_fidelidade, on=["id", "idLegislatura"], how="left")
    .fillna(0)
    .drop_duplicates()
)

In [517]:
weights = {
    "tempo_atuacao_percent": 1,
    "mesa_1": 5,
    "mesa_2": 3,
    "pos_lider": 4,
    "pos_comiss_pr": 3,
    "relatorias_ln": 2,
    "mand_dep_estadual":.2,
    "mand_ver": .1,
    "mand_sen": .7,
    "fid_municipais": 1,
    "fid_gerais": 1,
}

In [518]:
for var_name, weight in weights.items():
    df_final[f"dim_{var_name}"] = df_final[var_name] * weight

In [520]:
df_final.to_excel('df_final.xlsx')

In [519]:
df_final

,id,idLegislatura,ultimoStatus.nome,ultimoStatus.siglaPartido,sexo,dataNascimento,escolaridade,tempo_atuacao_dias,tempo_atuacao_percent,mesa_1,...,dim_mesa_1,dim_mesa_2,dim_pos_lider,dim_pos_comiss_pr,dim_relatorias_ln,dim_mand_dep_estadual,dim_mand_ver,dim_mand_sen,dim_fid_municipais,dim_fid_gerais
0,73909,52,OSVALDO BIOLCHI,PMDB,M,1934-12-16,Superior,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,6.182085,0.0,0.0,0.0,1,1
8,73984,52,MOISÉS LIPNIK,PDT,M,1953-08-05,Superior,136,0.09,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.0,1,0
16,73764,51,ABELARDO LUPION,DEM,M,1952-08-25,Superior Incompleto,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,1
17,73886,51,ADÃO PRETTO,PT,M,1945-12-18,Primário,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.0,1,1
18,74328,51,ADAUTO PEREIRA,PFL,M,1935-06-04,Superior,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8375,160632,55,Zé Silva,SOLIDARIEDADE,M,1963-05-11,Pós-Graduação,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,6.270988,0.0,0.0,0.0,1,1
8376,178923,55,Zeca Cavalcanti,PTB,M,1966-07-21,Pós-Graduação,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,4.394449,0.0,0.0,0.0,1,1
8377,160592,55,Zeca Dirceu,PT,M,1978-06-21,Superior,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,3.891820,0.0,0.0,0.0,1,1
8378,178902,55,ZECA DO PT,PT,M,1950-02-24,Superior,1460,1.00,0.0,...,0.0,0.0,0.0,0.0,4.795791,0.0,0.0,0.0,1,1
